In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.1.0'

In [3]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

In [4]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 2.1.0


In [5]:
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

C:\anaconda3\envs\auto_retina\lib\site-packages\ipykernel_launcher.py:3: UserWarning: No GPU found. Please ensure you have installed TensorFlow correctly
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color


In [7]:
from keras.models import load_model
from keras_retinanet import models

model_path = os.path.join('keras-retinanet/snapshots', sorted(os.listdir('keras-retinanet/snapshots'), reverse=True)[0])
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

labels_to_names = pd.read_csv('data/classes.csv', header=None).T.loc[0].to_dict()

snapshots\resnet50_csv_50.h5
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [

In [8]:
test_df = pd.read_csv("data/annotations_test.csv")
test_df.head()

,data/hand_085.png,114,27,218,223,hand
0,data/hand_111.png,38,50,117,220,hand
1,data/hand_003.png,1,77,59,223,hand
2,data/hand_128.png,44,58,150,217,hand
3,data/hand_006.png,54,49,146,218,hand
4,data/hand_012.png,79,37,167,220,hand


In [9]:
import skimage.io as io

def predict(image):
    image = preprocess_image(image.copy())
    image, scale = resize_image(image)

    boxes, scores, labels = model.predict_on_batch(
     np.expand_dims(image, axis=0)
     )

    boxes /= scale

    return boxes, scores, labels

umbralScore = 0.95

def draw_detections(image, boxes, scores, labels):
      for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < umbralScore:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(image, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(image, b, caption)

In [10]:
for index, row in test_df.iterrows():
    print(row[0], index)
    image = io.imread(row[0])
    #image = io.imread('images/hand2.jpg')

    boxes, scores, labels = predict(image)

    draw = image.copy()
    draw_detections(draw, boxes, scores, labels)

    plt.axis('off')
    plt.imshow(draw)
    plt.show()

data/hand_111.png 0


NameError: name 'model' is not defined

In [11]:
# get the reference to the webcam
camera = cv2.VideoCapture(0)
camera_height = 500

while(True):
    # read a new frame
    _, frame = camera.read()
    
    # flip the frameq
    frame = cv2.flip(frame, 1)

    # rescaling camera output
    #aspect = frame.shape[1] / float(frame.shape[0])
    #res = int(aspect * camera_height) # landscape orientation - wide image
    #frame = cv2.resize(frame, (res, camera_height))
    
    boxes, scores, labels = predict(frame)

 
    draw = frame.copy()
   # draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    #draw_detections(draw, boxes, scores, labels)

    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score > 0.8:
            print(box)
            b = box.astype(int)
            color = label_color(label)
            draw_box(draw, b, color=color)
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            
    #plt.axis('off')
    #plt.imshow(draw)
    #plt.show()

    # show the frame
    cv2.imshow("Test out", draw)
    #cv2.imshow("Test out2", roi)

    key = cv2.waitKey(1)

    # quit camera if 'q' key is pressed
    if key & 0xFF == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'copy'

In [12]:
camera.release()